# Road Following 

If you've run through the collision avoidance sample, your should be familiar following four steps:

1. Data collection
2. Training
3. Optimizing
3. Deployment

In this notebook, we'll do the exact same thing! Except, instead of classification, you'll learn a different fundamental technique known as **regression**, which we'll use to
enable JetBot to follow a path.

This notebook is for **Data Collection** and we will be performing the following tasks:

1. Display the live camera feed from the robot
2. Create and connect to a gamepad controller
3. Finally collect data by performing the following steps
- Place the JetBot in different positions on a path (offset from center, different angles, etc)
- Using a gamepad controller, place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.
- Store the X, Y values of this green dot along with the image from the robot's camera

What we get, is a 'carrot on a stick' that moves along our desired trajectory.  Deep learning decides where to place the carrot, and JetBot just follows it :)

### Labeling example video

Execute the block of code below to see an example of how to label the images.

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

### Import Libraries

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming. 

In [ ]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed

Firstly, let's initialize and display our camera like we did in the previous notebooks.

Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset. In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [ ]:
camera = Camera.instance(width=224, height=224)
image_widget = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

target_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)      # cv2.circle(image, center_coordinates, radius, color (BGR), thickness)
    image = cv2.line(image, (x, y), (112,244), (255,0,0), 3)  # cv2.line(image, start_point, end_point, color (BGR), thickness)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

### Create Gamepad Controller

This step is similar to "Remote Control" task. In this task, we will use a gamepad controller to label our images.

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [ ]:
controller = widgets.Controller(index=0)

display(controller)

### Connect Gamepad Controller to Label Images

Now, even though we've connected our gamepad, we haven't yet attached the controller to label images! We'll connect that to the left and right vertical axes using the dlink function.

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (y_slider, 'value'))

### Collect data

The following block of code will display the live image feed, as well as the number of images we've saved.  We store
the target X, Y values by

1. Place the green dot on the target
2. Press 'down' on the DPAD to save

This will store a file in the ``dataset_xy`` folder with files named

``xy_<x value>_<y value>_<uuid>.jpg``

When we train, we load the images and parse the x, y values from the filename

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

# for b in controller.buttons:
#     b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[0].observe(save_snapshot, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

# Save dataset in a zip

Once you've collected enough data, we can choose to copy that data to a GPU desktop or cloud machine for training. To do that, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q road_following.zip {DATASET_DIR}

You should see a file named ``road_following.zip`` in the Jupyter Lab file browser.

Next, Let's move on to the "train_model.ipynb" notebook so that we can start training our model on the collected dataset!